In [ ]:
import random
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy import stats
from tqdm import tqdm

class HillAnt():
    '''
    HillAnt class for simulating a 2D random walk.
    '''

    def __init__(self):
        self.reset()

    def reset(self):
        self.pos_x = 0
        self.pos_y = 0
        self.seconds = 0
    
    def step(self):
        direction = random.choice([1, -1])
        is_y = random.choice([True, False])
        if is_y:
            self.pos_y += direction * 10
        else:
            self.pos_x += direction * 10
        self.seconds += 1

    @property
    def coordinates(self):
        return [self.pos_x, self.pos_y]
    
    
    def satisfy_equation(self, eq):
        def cond():
            if(eq(self.coordinates[0], self.coordinates[1])):
                return True
            return False
        return cond

    
    def sim(self, condition):
        def run():
            sim_result = 0
            while not condition() and self.seconds<= 1000000:
                self.step()
            sim_result = self.seconds
            self.reset()
            return sim_result
        return run
    
    def sim_fixed_run(self, condition, seconds):
        def run():
            for i in range(seconds+1):
                if condition():
                    self.reset()
                    return True
                self.step()
            self.reset()
            return False
        return run

   







In [ ]:
#define function to run simulation
def monteCarlo(sim, n):
    results = []
    for _ in  tqdm(range(0, n)):
        result = sim()
        results.append(result)
    return results

#function to analyze results
def analyze(res):
    
    print(f'mean:  {np.mean(res)}')
    print(f'median: {np.median(res)}')
    print(f'mode: {stats.mode(res)}')

    print(f'Means Confidence Intervals: {st.t.interval(0.95, len(res)-1, loc=sum(res)/len(res), scale=st.sem(res))}')

In [ ]:
hillAnt = HillAnt()

#number of runs
n2 = 1000000 #for line boundary
n = 100000

#define the boundary conditions for the simulation
def boundary3(x, y):
    return (((x-2.5)/30)**2) + (((y-2.5)/40)**2) > 1

def square_boundary(x, y):
    return abs(x) >= 20 or abs(y)>= 20

def line(x, y):
    return (-1*x) + 10 == y

def line2(x, y):
    return (-1*x) + 20 == y

In [ ]:
#run simulations and gather results
res1 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(boundary3)), n)
analyze(res1)

res2 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(square_boundary)), n)
analyze(res2)

res3 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(line)), n2)
analyze(res3)

In [ ]:
results = []
summation = 0
for i in range(1, 31, 2):
    res4 = monteCarlo(hillAnt.sim_fixed_run(hillAnt.satisfy_equation(line), i), n)
    rs =sum(res4)/len(res4)
    summation+=i*rs
    print(summation)
    results.append(rs)

In [ ]:
sum(res4)/len(res4)

In [ ]:
prob = []
for i in tqdm(range(0,1000000)):
    prob.append(np.count_nonzero(res3 == i)/len(res3))

In [ ]:
plt.plot(prob)
plt.xlim([0, 100])

In [ ]:
def generate_random_walk(n):
    if n == 0:
        return []
    x, y = 0, 0  # Starting position
    paths = []
    directions = [(10, 0), (-10, 0), (0, 10), (0, -10)]  # Right, Left, Up, Down

    def explore_path(current_path, current_x, current_y):
        if len(current_path) == n:
            paths.append(current_path)
            return

        for dx, dy in directions:
            next_x, next_y = current_x + dx, current_y + dy
            next_path = current_path + [(next_x, next_y)]
            explore_path(next_path, next_x, next_y)

    explore_path([(0, 0)], x, y)
    return paths

def has_condition(path, cond):
    for point in path:
        if cond(point[0], point[1]):
            return True
    return False

def has_condition_last(path, cond):
    point = path[-1]
    if cond(point[0], point[1]):
        return True
    return False

def numerate_paths(paths, cond):
    conds = []
    for path in paths:
        conds.append(has_condition_last(path, cond))
    return conds



In [ ]:
prob=1
for i in range(1, 15):
    probs = []
    paths = generate_random_walk(i)
    b = np.asarray(numerate_paths(paths, line))
    prob *= i*sum(b)/len(b)
    print(prob)
    probs.append(prob)

In [1]:
def formula(n):
    if n%2 == 0:
        return 0
    
    if n == 1:
        return 0.5
    
    return formula(n-2) * (1 - (1/(n+1)))


In [ ]:
combined = 0
for i in range(1, 1111, 2):
    combined =(formula(i))
    print(combined)